In [58]:
from sklearn import datasets
import numpy as np
import pandas as pd
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler

data = datasets.load_boston()
X = pd.DataFrame(data.data,columns=data.feature_names).values
y= data.target
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [59]:
X.shape

(506, 13)

In [90]:
class SIregression:
  def __init__(self,num_particles=10,learning_rate=0.0003,batch_size=10,epoch=10,inertia = 0.9, c1 = 0.5, c2 = 0.4):
    self.lr_init = learning_rate
    self.batch_size = batch_size
    self.num_particles = num_particles
    self.trained_weights = None
    self.epoch = epoch
    self.c2 = c2
    self.c1 = c1
    self.inertia = inertia

  def split(self,data):
    # one can override this function build custom data spliter (for temporal data)
    split_percentage = 1
    # returns training data,validation data
    return data[:int(data.shape[0]*split_percentage)],data[int(data.shape[0]*(split_percentage)):] 

  def loss_func(self,target,values):
    # one can overrided this function to create a custome lossfunction
    return np.sum((target-values)**2)/target.shape[0]

  def exp_decay(self,loss):
    # learning rate will decay with respect to the loss
    k = 0.3
    lrate = self.lr_init * np.exp(k*(loss-self.thresh))
    return lrate

  def velocity_update(self,v1,pbest,gbest,present):
    #updating the velocity using the formula
    # logistic equation of chaos as r > 4 
    self.inertia = 4*self.inertia*(1-self.inertia)
    return self.inertia*v1 + self.c1*np.random.rand()*(pbest-present)+self.c2*np.random.rand()*(gbest-present) 

  def fit(self,X,y):
    # initializing starting position, velocity, particles best values
    particles=[]
    best_loss = 0
    global_best = None
    # adding one for the bias term
    X = np.append(X,np.ones(shape = (X.shape[0],1)),axis=1)

    for particle in norm.rvs(scale = 5, size = (self.num_particles , X.shape[1])):
      particles.append({'values':particle,
                        'velocity':np.random.randn(X.shape[1]),
                        'present_best': particle})
      
      if len(particles) == 1:
        global_best = particle
        best_loss = self.loss_func(y[:self.batch_size],np.array([np.sum(particle*x) for x in X[:self.batch_size]]))
      else:
        current_loss = self.loss_func(y[:self.batch_size],np.array([np.sum(particle*x) for x in X[:self.batch_size]]))
        if current_loss<best_loss:
          global_best = particle
          best_loss = current_loss
    particles = np.array(particles)

    #algorithm
    ep = 0
    loss = 9999
    while self.epoch>ep:
      train_data,val_data = self.split(np.append(X,np.vstack(y),axis=1))
      previous_loss = self.loss_func(train_data[:,-1],np.array([np.sum(global_best*x) for x in train_data[:,:-1]]))
      for i,particle in enumerate(particles):
        # print(particle.values)
        particles[i]['velocity'] = self.velocity_update(particle['velocity'],global_best,particle['present_best'],particles[i]['values'])
        
        # updating the position
        particles[i]['values'] = particles[i]['values'] + particles[i]['velocity']

        # updating present best and global best
        present_loss = self.loss_func(train_data[:,-1],np.array([np.sum(particles[i]['values']*x) for x in train_data[:,:-1]]))
        present_best_loss = self.loss_func(train_data[:,-1],np.array([np.sum(particles[i]['present_best']*x) for x in train_data[:,:-1]]))
        global_best_loss = self.loss_func(train_data[:,-1],np.array([np.sum(global_best*x) for x in train_data[:,:-1]]))
        
        if present_loss < present_best_loss:
          particles[i]['present_best'] = particles[i]['values']
        if present_best_loss < global_best_loss:
          global_best = particles[i]['present_best']
        
      loss = self.loss_func(train_data[:,-1],np.array([np.sum(global_best*x) for x in train_data[:,:-1]]))
      # mutation for local minima purpose
      if int(previous_loss) == int(loss):
        for i,_ in enumerate(particles):
          # shift the point by a randnom distance which decreases as the iterations reaches end
          particles[i]['values'] = particles[i]['values'] + norm.rvs(scale =(self.epoch-ep)/self.epoch if (self.epoch-ep)/self.epoch > 0.2 else 0.2 ,size=(len(particles[i]['values'])))
      
      print("training loss {} of epoch {}".format(loss,ep+1))
      ep = ep+1
    
    print(global_best.shape)
    self.trained_weights = global_best
    return self.trained_weights

  def predict(self,X):
    # adding one for the bias term
    print(X.shape)
    X = np.append(X,np.ones(shape = (X.shape[0],1)),axis=1)
    return np.array([np.sum(self.trained_weights*x) for x in X[:,:]])

In [92]:
model = SIregression(num_particles=70,epoch=50)
model.fit(X[:-50],y[:-50])
print(model.loss_func(y[-50:],model.predict(X[-50:])))

training loss 154.5629408829471 of epoch 1
training loss 154.5629408829471 of epoch 2
training loss 137.02750907496628 of epoch 3
training loss 105.03039722149933 of epoch 4
training loss 105.03039722149933 of epoch 5
training loss 81.20740329737849 of epoch 6
training loss 74.57354573706556 of epoch 7
training loss 74.57354573706556 of epoch 8
training loss 72.74709778176567 of epoch 9
training loss 67.85406094575369 of epoch 10
training loss 66.08700702495035 of epoch 11
training loss 59.43497181341694 of epoch 12
training loss 53.719489654740485 of epoch 13
training loss 52.31239671420372 of epoch 14
training loss 50.06149107410644 of epoch 15
training loss 50.06149107410644 of epoch 16
training loss 43.52470940839406 of epoch 17
training loss 42.48568453954056 of epoch 18
training loss 41.580862780287276 of epoch 19
training loss 38.668752681556775 of epoch 20
training loss 35.86356035302212 of epoch 21
training loss 35.86356035302212 of epoch 22
training loss 34.60426791518303 of 

In [14]:
from sklearn.linear_model import LinearRegression
mod = LinearRegression()
mod.fit(X[:-50],y[:-50])
print(model.loss_func(y[-50:],mod.predict(X[-50:])))

10.96041067942274


In [15]:
mod.coef_

array([-0.90758523,  1.1448883 ,  0.22037032,  0.6376401 , -2.04067241,
        2.67970135,  0.29749879, -3.02099961,  3.14455125, -2.60489475,
       -1.97322935,  0.89921549, -3.96002849])

In [53]:
model.trained_weights

array([-0.71736973,  0.86814684,  0.07047775,  0.73234371, -1.47926267,
        2.77617494,  0.1899308 , -3.34406331,  0.93882257, -1.12369738,
       -2.30365275,  1.10506165, -3.1393519 , 22.79075347])